# Projeto: Esgotamento Sanitário na Região Nordeste (2010)

### Autores:

- Gabriel Igor Queiroz Costa
- Erick de Oliveira SIlva

In [1]:
# Uploading files from your local file system

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving datasets.zip to datasets.zip
User uploaded file "datasets.zip" with length 18256 bytes


In [2]:
!unzip datasets.zip

Archive:  datasets.zip
  inflating: esg_SE.csv              
  inflating: esg_AL.csv              
  inflating: esg_BA.csv              
  inflating: esg_CE.csv              
  inflating: esg_MA.csv              
  inflating: esg_PB.csv              
  inflating: esg_PE.csv              
  inflating: esg_PI.csv              
  inflating: esg_RN.csv              


In [0]:
!pip install folium shapely

In [0]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random

# 1 - Introdução 

O Instituto Brasileiro de Geografia e Estatística disponibiliza em seu site diversos dados relativos ao Brasil. Nesse notebook nós acessamos dados referentes  ao percentual de esgotamento sanitário de cada um dos estados do Nordeste no ano de 2010 para gerar mapas temáticos usando o choropleth e analisar se existem grandes diferenças entre o estado dessa mesma região.

## 1.1 Importando os datasets

In [0]:
dataRN = pd.read_csv("esg_RN.csv")
dataPB = pd.read_csv("esg_PB.csv")
dataPE = pd.read_csv("esg_PE.csv")
dataCE = pd.read_csv("esg_CE.csv")
dataMA = pd.read_csv("esg_MA.csv")
dataPI = pd.read_csv("esg_PI.csv")
dataAL = pd.read_csv("esg_AL.csv")
dataSE = pd.read_csv("esg_SE.csv")
dataBA = pd.read_csv("esg_BA.csv")

Após importar os dados, podemos concatená-los em um único dataset:

In [0]:
data = pd.concat([dataRN, dataPB, dataPE, dataCE, dataMA, dataPI, dataAL, dataSE, dataBA], ignore_index=True)

Para alterar o nome das colunas:

In [0]:
data.columns = ["Cidade", "Esgotamento"]

# 2 -  Importing GeoJson files

In [0]:
# searching the files in geojson/geojs-xx-mun.json
br_states = os.path.join('geojson', 'geojs-24-mun.json')

# load the data and use 'latin-1'encoding because the accent
geo_json_data = json.load(open(br_states,encoding='latin-1'))

In [0]:
# http://cidades.ibge.gov.br/painel/historico.php?codmun=241030
# Presidente Juscelino city changes your name to Serra Caiada
geo_json_data['features'][112]['properties']['description'] = 'Serra Caiada'
geo_json_data['features'][112]['properties']['name'] = 'Serra Caiada'

In [0]:
cities = []
# list all cities in the state
for city in geo_json_data['features']:
        cities.append(city['properties']['description'])
cities

In [125]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(geo_json_data).add_to(m)

m

# 3 - Drawing the Choropleth

In [126]:
df.head()

,Cidade,Esgotamento
0,Acari,81.4
1,Açu,13.3
2,Afonso Bezerra,51.3
3,Água Nova,7.1
4,Alexandria,56.1


In [127]:
# colormap yellow and green (YlGn)
colormap = linear.YlGn_03.scale(
    df["Esgotamento"].min(),
    df["Esgotamento"].max())

print(colormap(5000.0))

colormap

#31a354


In [128]:
# Converter para dicionário
esg_dict = df.set_index('Cidade')['Esgotamento']
esg_dict[:100]

Cidade
Acari                      81.4
Açu                        13.3
Afonso Bezerra             51.3
Água Nova                   7.1
Alexandria                 56.1
Almino Afonso              12.8
Alto do Rodrigues           8.4
Angicos                    38.3
Antônio Martins            24.6
Apodi                      10.5
Areia Branca               29.4
Arês                       20.8
Augusto Severo             16.6
Baía Formosa               10.1
Baraúna                     9.8
Barcelona                  23.7
Bento Fernandes            14.4
Bodó                       40.9
Bom Jesus                  10.8
Brejinho                   32.3
Caiçara do Norte           40.5
Caiçara do Rio do Vento    56.8
Caicó                      76.3
Campo Redondo              43.5
Canguaretama                5.4
Caraúbas                   12.7
Carnaúba dos Dantas        16.3
Carnaubais                 45.3
Ceará-Mirim                54.5
Cerro Corá                  4.8
                           ... 
L

In [130]:
# option 2

# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# create a threshold of legend
threshold_scale = np.linspace(df['Esgotamento'].min(),
                              df['Esgotamento'].max(), 6, dtype=int).tolist()


m.choropleth(
    geo_data=geo_json_data,
    data=df,
    columns=['Cidade', 'Esgotamento'],
    key_on='feature.properties.description',
    fill_color='YlGn',
    legend_name='Esgotamento sanitário adequado (2010)',
    highlight=True,
    threshold_scale = threshold_scale
)

m